In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

In [2]:
# read comments cvs
# df_comments = pd.read_csv("Manual checking - comments.csv")
# df_comments = pd.read_csv("./data/pipeline/cq_parent_feats.csv")
df_comments = pd.read_csv("./data/pipeline/cq_parent_feats.csv")
df_comments

,Id,PostId,Score,Text,CreationDate,UserId,ContentLicense,of_answer,cnt_keywords,subject,...,ViewCount,LastEditDate,LastActivityDate,Title,Tags,AnswerCount,CommentCount,ClosedDate,FavoriteCount,CommunityOwnedDate
0,1027927,1204191,0,Do you have a contact over at Amazon? They nee...,2009-07-30T05:38:18.620,76337,CC BY-SA 2.5,False,1,NaN,...,10400.0,2009-08-03T03:09:04.357,2009-09-05T04:11:20.040,How to sign an Amazon web service request in ....,<c#><wcf><web-services><soap><amazon-web-servi...,4.0,2.0,NaN,3.0,NaN
1,2301473,1496066,4,This question and answers are out of date now ...,2010-02-25T02:48:51.043,50277,CC BY-SA 2.5,False,1,question,...,4772.0,2013-01-06T13:29:42.057,2018-11-17T00:09:24.550,Amazon SimpleDB Woes: Implementing counter att...,<amazon-web-services><amazon-simpledb>,6.0,1.0,NaN,7.0,NaN
2,2301480,327169,1,This question and answers are out of date now ...,2010-02-25T02:51:10.927,50277,CC BY-SA 2.5,False,1,question,...,959.0,NaN,2008-11-29T13:39:08.123,"How far can you really go with ""eventual"" cons...",<sql><database><amazon-web-services><amazon-si...,2.0,1.0,NaN,4.0,NaN
3,8008147,6735850,0,"@house9, I've looked at heroku already, and th...",2011-07-19T22:43:03.567,152092,CC BY-SA 3.0,False,1,I,...,2907.0,NaN,2013-03-19T11:17:48.443,Rails 3 Server on EC2,<ruby-on-rails-3><amazon-ec2><ruby-on-rails-3.1>,3.0,3.0,NaN,3.0,NaN
4,9594466,7864988,0,UIDevice `uniqueIdentifier` is deprecated in i...,2011-10-23T08:13:47.107,18091,CC BY-SA 3.0,False,1,UIDevice,...,783.0,NaN,2011-10-23T09:05:42.187,ios 5 AWS SDK S3PutObjectRequest,<amazon-s3><amazon-web-services><ios5>,1.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,111881787,63241324,0,@BenT We are thinking of using timestamps to i...,2020-08-05T17:36:12.297,2184946,CC BY-SA 4.0,False,1,NaN,...,21.0,NaN,2020-08-11T20:48:26.050,How to avoid retrieving outdated data from AWS...,<amazon-web-services><iot><aws-iot>,1.0,2.0,NaN,NaN,NaN
367,112537429,63640052,0,the only error thats in there is this warning ...,2020-08-28T20:32:43.217,14184500,CC BY-SA 4.0,False,1,Buffer,...,34.0,2020-08-28T22:39:27.823,2020-08-28T22:39:27.823,S3.upload not working in production environmen...,<node.js><amazon-web-services><amazon-s3><netl...,0.0,10.0,NaN,NaN,NaN
368,112577969,63644158,0,"Yes, using a load balancer would be ideal, but...",2020-08-31T01:47:22.793,12564768,CC BY-SA 4.0,False,1,NaN,...,19.0,2020-09-02T01:46:24.570,2020-09-02T01:46:24.570,ECS randomly terminates instances and starts a...,<amazon-ec2><amazon-ecs>,0.0,3.0,NaN,NaN,NaN
369,112606415,63679395,2,Do you absolutely need a 0.11 compatible answe...,2020-08-31T23:46:36.197,2291321,CC BY-SA 4.0,False,1,0.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_comments['from_dataset']='comments_que'

In [4]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371 entries, 0 to 370
Data columns (total 29 columns):
Id                    371 non-null int64
PostId                371 non-null int64
Score                 371 non-null int64
Text                  371 non-null object
CreationDate          371 non-null object
UserId                371 non-null object
ContentLicense        371 non-null object
of_answer             371 non-null bool
cnt_keywords          371 non-null int64
subject               127 non-null object
punctuation           371 non-null object
negative_statement    371 non-null int64
sentence              371 non-null object
subj_irrel            371 non-null bool
include_irrel         371 non-null bool
Id_que                355 non-null float64
CreationDate_que      355 non-null object
Score_que             355 non-null float64
ViewCount             355 non-null float64
LastEditDate          181 non-null object
LastActivityDate      355 non-null object
Title                

In [5]:
df_comments['Id_que']

0       1204191.0
1       1496066.0
2        327169.0
3       6735850.0
4       7864988.0
          ...    
366    63241324.0
367    63640052.0
368    63644158.0
369           NaN
370    63695581.0
Name: Id_que, Length: 371, dtype: float64

In [6]:
# df_answers = pd.read_csv("./data/pipeline/ca_parent_feats.csv")
df_answers = pd.read_csv("./data/pipeline/ca_parent_feats.csv")
# df_answers.columns
df_answers = df_answers.drop('Body',axis = 1)
# df_answers['Id_ans']==df_answers['Id']

In [7]:
df_comments['from_dataset']='comments_ans'

In [8]:
df_answers.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId',
       'ContentLicense', 'of_answer', 'cnt_keywords', 'subject', 'punctuation',
       'negative_statement', 'sentence', 'subj_irrel', 'include_irrel',
       'Id_ans', 'ParentId', 'CreationDate_ans', 'Score_ans',
       'LastActivityDate_ans', 'CommentCount_ans', 'Id_que',
       'CreationDate_que', 'Score_que', 'ViewCount', 'LastEditDate',
       'LastActivityDate_que', 'Title', 'Tags', 'AnswerCount',
       'CommentCount_que', 'ClosedDate', 'FavoriteCount',
       'CommunityOwnedDate'],
      dtype='object')

In [9]:
df_answers_a = pd.read_csv("./data/pipeline/a_parent_feats.csv")
df_answers_a = df_answers_a.drop('Body',axis = 1)

In [10]:
df_answers_a['from_dataset']='answers'

In [11]:
df_answers_a.columns

Index(['Id', 'PostTypeId', 'ParentId', 'CreationDate', 'Score', 'OwnerUserId',
       'OwnerDisplayName', 'LastActivityDate', 'CommentCount',
       'ContentLicense', 'cnt_keywords', 'subject', 'punctuation',
       'negative_statement', 'sentence', 'subj_irrel', 'include_irrel',
       'Id_que', 'CreationDate_que', 'Score_que', 'ViewCount', 'LastEditDate',
       'LastActivityDate_que', 'Title', 'Tags', 'AnswerCount',
       'CommentCount_que', 'ClosedDate', 'FavoriteCount', 'from_dataset'],
      dtype='object')

In [12]:
df_answers_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 30 columns):
Id                      598 non-null int64
PostTypeId              598 non-null int64
ParentId                598 non-null int64
CreationDate            598 non-null object
Score                   598 non-null int64
OwnerUserId             598 non-null object
OwnerDisplayName        598 non-null object
LastActivityDate        598 non-null int64
CommentCount            598 non-null object
ContentLicense          1 non-null object
cnt_keywords            598 non-null int64
subject                 185 non-null object
punctuation             598 non-null object
negative_statement      598 non-null int64
sentence                598 non-null object
subj_irrel              598 non-null bool
include_irrel           598 non-null bool
Id_que                  598 non-null int64
CreationDate_que        598 non-null object
Score_que               598 non-null int64
ViewCount               598 no

In [13]:
df_comments.columns

Index(['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId',
       'ContentLicense', 'of_answer', 'cnt_keywords', 'subject', 'punctuation',
       'negative_statement', 'sentence', 'subj_irrel', 'include_irrel',
       'Id_que', 'CreationDate_que', 'Score_que', 'ViewCount', 'LastEditDate',
       'LastActivityDate', 'Title', 'Tags', 'AnswerCount', 'CommentCount',
       'ClosedDate', 'FavoriteCount', 'CommunityOwnedDate', 'from_dataset'],
      dtype='object')

In [14]:
set(df_comments.columns)-set(df_answers.columns)

{'CommentCount', 'LastActivityDate', 'from_dataset'}

In [15]:
set(df_answers.columns)-set(df_comments.columns)

{'CommentCount_ans',
 'CommentCount_que',
 'CreationDate_ans',
 'Id_ans',
 'LastActivityDate_ans',
 'LastActivityDate_que',
 'ParentId',
 'Score_ans'}

In [16]:
print(len(df_comments.columns))
print(len(df_answers.columns))

29
34


In [17]:
df = pd.concat([df_comments,df_answers],axis = 0,ignore_index = True)

/Users/jianghuimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [18]:
df = pd.concat([df,df_answers_a],axis = 0,ignore_index = True)

/Users/jianghuimin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546 entries, 0 to 1545
Data columns (total 40 columns):
AnswerCount             1295 non-null float64
ClosedDate              29 non-null object
CommentCount            953 non-null object
CommentCount_ans        342 non-null object
CommentCount_que        940 non-null float64
CommunityOwnedDate      0 non-null float64
ContentLicense          949 non-null object
CreationDate            1546 non-null object
CreationDate_ans        342 non-null object
CreationDate_que        1295 non-null object
FavoriteCount           461 non-null float64
Id                      1546 non-null int64
Id_ans                  342 non-null float64
Id_que                  1295 non-null float64
LastActivityDate        953 non-null object
LastActivityDate_ans    342 non-null float64
LastActivityDate_que    940 non-null object
LastEditDate            630 non-null object
OwnerDisplayName        598 non-null object
OwnerUserId             598 non-null object
Paren

In [20]:
#CommunityOwnedDate      0 non-null float64
#drop the columns completely lost
df = df.drop('CommunityOwnedDate',axis = 1)

In [23]:
def Find(string):
    if string!=string:
        return 0
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)
    url_list=[x[0] for x in url]
    if len(url_list) != 0:
        return 1
    else:
        return 0
df['share_link']= df['Text'].apply(lambda x:Find(x))

In [24]:
def get_tags(tags):
#     print(tags)
    if tags != tags:
        return None
    else:
        tags = tags[1:-1]
        tags = tags.split('><')
        res = []
        for item in tags:
            if ('aws' in item) or ('amazon' in item):
                if item != 'aws' and item !='amazon-web-services':
                    res.append(item)
        if res == []:
            return None
        return res
df['aws_related_tags']= df['Tags'].apply(lambda x:get_tags(x))

In [25]:
def count_tag_num(tags):
    if tags == None:
        return None
    return len(tags)
df['tag_num']= df['aws_related_tags'].apply(lambda x:count_tag_num(x))

In [26]:
pd.unique(df['tag_num'])

array([nan,  1.,  2.,  3.,  4.])

In [27]:
def get_tags(tags):
    if tags != tags:
        return None
    else:
        tags = tags[1:-1]
        tags = tags.split('><')
        res = []
        for item in tags:
            if ('aws' in item) or ('amazon' in item):
                if item != 'aws' and item !='amazon-web-services':
                    res.append(item)
        if res == []:
            return None
        return res
df['aws_related_tags']= df['Tags'].apply(lambda x:get_tags(x))

In [28]:
def get_code(text):
    if text!=text:
        return 0
    code = re.findall('`([^"]*)`',text)
    if code ==[]:
        return 0
    else:
        return 1
df['share_code']= df['Text'].apply(lambda x:get_code(x))

In [30]:
df

,AnswerCount,ClosedDate,CommentCount,CommentCount_ans,CommentCount_que,ContentLicense,CreationDate,CreationDate_ans,CreationDate_que,FavoriteCount,...,negative_statement,of_answer,punctuation,sentence,subj_irrel,subject,share_link,aws_related_tags,tag_num,share_code
0,4.0,NaN,2,NaN,NaN,CC BY-SA 2.5,2009-07-30T05:38:18.620,NaN,2009-07-30T02:48:06.287,3.0,...,1,False,.,They need to be made to understand why they sh...,False,NaN,0,None,NaN,0
1,6.0,NaN,1,NaN,NaN,CC BY-SA 2.5,2010-02-25T02:48:51.043,NaN,2009-09-30T04:04:17.807,7.0,...,0,False,.,This question and answers are outdated now tha...,False,question,1,[amazon-simpledb],1.0,0
2,2.0,NaN,1,NaN,NaN,CC BY-SA 2.5,2010-02-25T02:51:10.927,NaN,2008-11-29T04:36:13.523,4.0,...,0,False,.,This question and answers are outdated now tha...,False,question,1,[amazon-simpledb],1.0,0
3,3.0,NaN,3,NaN,NaN,CC BY-SA 3.0,2011-07-19T22:43:03.567,NaN,2011-07-18T16:06:30.553,3.0,...,0,False,.,"Regarding the other link, I've looked them up,...",False,I,0,[amazon-ec2],1.0,0
4,1.0,NaN,2,NaN,NaN,CC BY-SA 3.0,2011-10-23T08:13:47.107,NaN,2011-10-23T07:47:23.067,NaN,...,0,False,.,UIDevice `uniqueIdentifier` is deprecated in i...,False,UIDevice,0,[amazon-s3],1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,2.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-17T22:58:56.913,NaN,2020-08-17T17:34:48.637,NaN,...,0,NaN,.,>Also your <strong>platform version is outdate...,False,NaN,0,"[amazon-elastic-beanstalk, amazon-cloudformation]",2.0,0
1542,2.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-22T03:52:15.740,NaN,2020-08-21T16:07:57.017,NaN,...,0,NaN,it,"PS: I thought EKS 1.13 is already deprecated, ...",False,NaN,0,[amazon-eks],1.0,0
1543,7.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-25T05:41:56.313,NaN,2020-05-19T14:36:23.437,1.0,...,0,NaN,older,it deprecated older,False,NaN,0,None,NaN,0
1544,1.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-27T14:22:53.153,NaN,2020-08-27T14:15:51.063,NaN,...,0,NaN,code,rather than the deprecated <code,False,NaN,0,[amazon-s3],1.0,0


In [31]:
# read answers cvs
# df_answers = pd.read_csv("Manual checking - answers.csv")
df_answers_checked = pd.read_csv("./data/pipeline/Manual checking - answers (1).csv")
df_answers_checked = df_answers_checked[:100]

In [32]:
df_answers_checked = df_answers_checked[['answer_id','outdated (manually checked)']]

In [33]:
df = df.merge(df_answers_checked,left_on='Id_ans',right_on='answer_id',how = 'left')

In [34]:
# read comments cvs
# df_comments = pd.read_csv("Manual checking - comments.csv")
df_comments_checked = pd.read_csv("./data/pipeline/Manual checking - comments(new).csv")
df_comments_checked = df_comments_checked[:194]


In [35]:
# df_comments_checked 

In [36]:
df_comments_checked = df_comments_checked[['Id','outdated (manually checked)']]

In [37]:
df = df.merge(df_comments_checked,left_on='Id',right_on='Id',how = 'left')

In [38]:
def get_outdated(x,y):
    if x == x:
        return x
    elif y == y:
        return y
    else:
        return None
df['outdated (manually checked)']= df.apply(lambda x:get_outdated(x['outdated (manually checked)_x'],x['outdated (manually checked)_y']),axis =1)

In [39]:
def train_or_test(manual):
    if manual == None:
        return 'Test'
    else:
        return 'Train'
df['is_training']= df['outdated (manually checked)'].apply(lambda x:train_or_test(x))
 

In [40]:
df = df.drop(['outdated (manually checked)_x','outdated (manually checked)_y','answer_id'],axis = 1)

In [41]:
df = df.drop(['tag_num'],axis =1)

In [43]:
def get_keywords(text):
    if text != text:
        return None
    text_dict = ['outdated','deprecated','obsolete','out of date','not deprecated','old version','probably outdated','remove','instead']
    res = []
    for item in text_dict:
        if item in text:
            res.append(item)
    res = set(res)
    if 'not deprecated' in res and 'deprecated' in res:
        res.remove('deprecated')
    return res
df['keywords']= df['Text'].apply(lambda x:get_keywords(x))

In [45]:
df.columns

Index(['AnswerCount', 'ClosedDate', 'CommentCount', 'CommentCount_ans',
       'CommentCount_que', 'ContentLicense', 'CreationDate',
       'CreationDate_ans', 'CreationDate_que', 'FavoriteCount', 'Id', 'Id_ans',
       'Id_que', 'LastActivityDate', 'LastActivityDate_ans',
       'LastActivityDate_que', 'LastEditDate', 'OwnerDisplayName',
       'OwnerUserId', 'ParentId', 'PostId', 'PostTypeId', 'Score', 'Score_ans',
       'Score_que', 'Tags', 'Text', 'Title', 'UserId', 'ViewCount',
       'cnt_keywords', 'from_dataset', 'include_irrel', 'negative_statement',
       'of_answer', 'punctuation', 'sentence', 'subj_irrel', 'subject',
       'share_link', 'aws_related_tags', 'share_code',
       'outdated (manually checked)', 'is_training', 'keywords'],
      dtype='object')

In [48]:
df.to_csv("./data/pipeline/1550_rows_45_columns.csv")

In [47]:
df

,AnswerCount,ClosedDate,CommentCount,CommentCount_ans,CommentCount_que,ContentLicense,CreationDate,CreationDate_ans,CreationDate_que,FavoriteCount,...,punctuation,sentence,subj_irrel,subject,share_link,aws_related_tags,share_code,outdated (manually checked),is_training,keywords
0,4.0,NaN,2,NaN,NaN,CC BY-SA 2.5,2009-07-30T05:38:18.620,NaN,2009-07-30T02:48:06.287,3.0,...,.,They need to be made to understand why they sh...,False,NaN,0,None,0,None,Test,{obsolete}
1,6.0,NaN,1,NaN,NaN,CC BY-SA 2.5,2010-02-25T02:48:51.043,NaN,2009-09-30T04:04:17.807,7.0,...,.,This question and answers are outdated now tha...,False,question,1,[amazon-simpledb],0,None,Test,{out of date}
2,2.0,NaN,1,NaN,NaN,CC BY-SA 2.5,2010-02-25T02:51:10.927,NaN,2008-11-29T04:36:13.523,4.0,...,.,This question and answers are outdated now tha...,False,question,1,[amazon-simpledb],0,True,Train,{out of date}
3,3.0,NaN,3,NaN,NaN,CC BY-SA 3.0,2011-07-19T22:43:03.567,NaN,2011-07-18T16:06:30.553,3.0,...,.,"Regarding the other link, I've looked them up,...",False,I,0,[amazon-ec2],0,None,Test,{outdated}
4,1.0,NaN,2,NaN,NaN,CC BY-SA 3.0,2011-10-23T08:13:47.107,NaN,2011-10-23T07:47:23.067,NaN,...,.,UIDevice `uniqueIdentifier` is deprecated in i...,False,UIDevice,0,[amazon-s3],1,None,Test,{deprecated}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1545,2.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-17T22:58:56.913,NaN,2020-08-17T17:34:48.637,NaN,...,.,>Also your <strong>platform version is outdate...,False,NaN,0,"[amazon-elastic-beanstalk, amazon-cloudformation]",0,None,Test,None
1546,2.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-22T03:52:15.740,NaN,2020-08-21T16:07:57.017,NaN,...,it,"PS: I thought EKS 1.13 is already deprecated, ...",False,NaN,0,[amazon-eks],0,None,Test,None
1547,7.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-25T05:41:56.313,NaN,2020-05-19T14:36:23.437,1.0,...,older,it deprecated older,False,NaN,0,None,0,None,Test,None
1548,1.0,NaN,CC BY-SA 4.0,NaN,0.0,NaN,2020-08-27T14:22:53.153,NaN,2020-08-27T14:15:51.063,NaN,...,code,rather than the deprecated <code,False,NaN,0,[amazon-s3],0,None,Test,None
